# Azure Cognitive Services 工具包

此工具包用于与 `Azure Cognitive Services API` 交互，以实现一些多模态功能。

目前此工具包捆绑了四个工具：
- AzureCogsImageAnalysisTool：用于从图像中提取标题、对象、标签和文本（注意：此工具目前在 Mac OS 上不可用，因为其依赖的 `azure-ai-vision` 包目前仅支持 Windows 和 Linux）。
- AzureCogsFormRecognizerTool：用于从文档中提取文本、表格和键值对。
- AzureCogsSpeech2TextTool：用于将语音转换为文本。
- AzureCogsText2SpeechTool：用于将文本合成为语音。
- AzureCogsTextAnalyticsHealthTool：用于提取医疗保健实体。

首先，你需要设置一个 Azure 账户并创建一个 Cognitive Services 资源。你可以遵循 [此处](https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account?tabs=multiservice%2Cwindows) 的说明来创建资源。

然后，你需要获取资源的终结点（endpoint）、密钥（key）和区域（region），并将它们设置为环境变量。你可以在资源的“密钥和终结点”页面找到它们。

In [ ]:
%pip install --upgrade --quiet  azure-ai-formrecognizer > /dev/null
%pip install --upgrade --quiet  azure-cognitiveservices-speech > /dev/null
%pip install --upgrade --quiet  azure-ai-textanalytics > /dev/null

# For Windows/Linux
%pip install --upgrade --quiet  azure-ai-vision > /dev/null

In [ ]:
%pip install -qU langchain-community

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-"
os.environ["AZURE_COGS_KEY"] = ""
os.environ["AZURE_COGS_ENDPOINT"] = ""
os.environ["AZURE_COGS_REGION"] = ""

## 创建工具包

In [19]:
from langchain_community.agent_toolkits import AzureCognitiveServicesToolkit

toolkit = AzureCognitiveServicesToolkit()

In [ ]:
[tool.name for tool in toolkit.get_tools()]

['Azure Cognitive Services Image Analysis',
 'Azure Cognitive Services Form Recognizer',
 'Azure Cognitive Services Speech2Text',
 'Azure Cognitive Services Text2Speech']

## 在 Agent 中使用

In [20]:
from langchain.agents import AgentType, initialize_agent
from langchain_openai import OpenAI

In [21]:
llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
agent.run(
    "What can I make with these ingredients?"
    "https://images.openai.com/blob/9ad5a2ab-041f-475f-ad6a-b51899c50182/ingredients.png"
)



> Entering new AgentExecutor chain...

Action:
```
{
  "action": "Azure Cognitive Services Image Analysis",
  "action_input": "https://images.openai.com/blob/9ad5a2ab-041f-475f-ad6a-b51899c50182/ingredients.png"
}
```


Observation: Caption: a group of eggs and flour in bowls
Objects: Egg, Egg, Food
Tags: dairy, ingredient, indoor, thickening agent, food, mixing bowl, powder, flour, egg, bowl
Thought: I can use the objects and tags to suggest recipes
Action:
```
{
  "action": "Final Answer",
  "action_input": "You can make pancakes, omelettes, or quiches with these ingredients!"
}
```

> Finished chain.


'You can make pancakes, omelettes, or quiches with these ingredients!'

In [ ]:
audio_file = agent.run("Tell me a joke and read it out for me.")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Azure Cognitive Services Text2Speech",
  "action_input": "Why did the chicken cross the playground? To get to the other slide!"
}
```


Observation: /tmp/tmpa3uu_j6b.wav
Thought: I have the audio file of the joke
Action:
```
{
  "action": "Final Answer",
  "action_input": "/tmp/tmpa3uu_j6b.wav"
}
```

> Finished chain.


'/tmp/tmpa3uu_j6b.wav'

In [ ]:
from IPython import display

audio = display.Audio(audio_file)
display.display(audio)

In [22]:
agent.run(
    """The patient is a 54-year-old gentleman with a history of progressive angina over the past several months.
The patient had a cardiac catheterization in July of this year revealing total occlusion of the RCA and 50% left main disease ,
with a strong family history of coronary artery disease with a brother dying at the age of 52 from a myocardial infarction and
another brother who is status post coronary artery bypass grafting. The patient had a stress echocardiogram done on July , 2001 ,
which showed no wall motion abnormalities , but this was a difficult study due to body habitus. The patient went for six minutes with
minimal ST depressions in the anterior lateral leads , thought due to fatigue and wrist pain , his anginal equivalent. Due to the patient's
increased symptoms and family history and history left main disease with total occasional of his RCA was referred for revascularization with open heart surgery.

List all the diagnoses.
"""
)



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "azure_cognitive_services_text_analyics_health",
  "action_input": "The patient is a 54-year-old gentleman with a history of progressive angina over the past several months. The patient had a cardiac catheterization in July of this year revealing total occlusion of the RCA and 50% left main disease, with a strong family history of coronary artery disease with a brother dying at the age of 52 from a myocardial infarction and another brother who is status post coronary artery bypass grafting. The patient had a stress echocardiogram done on July, 2001, which showed no wall motion abnormalities, but this was a difficult study due to body habitus. The patient went for six minutes with minimal ST depressions in the anterior lateral leads, thought due to fatigue and wrist pain, his anginal equivalent. Due to the patient's increased symptoms and family history and history left main disease with total occasional of his RCA was re

'The text contains the following diagnoses: progressive angina, coronary artery disease, myocardial infarction, and coronary artery bypass grafting.'